## Deliverable 3. Create a Travel Itinerary Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [24]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Jamestown,US,72.99,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
1,3,East London,ZA,52.81,clear sky,-33.0153,27.9116,Tu Casa
2,4,Georgetown,MY,82.89,few clouds,5.4112,100.3354,Cititel Penang
3,5,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,7,Puerto Ayora,EC,72.55,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [25]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,42.0970,-79.2353
1,-33.0153,27.9116
2,5.4112,100.3354
3,22.0752,-159.3190
4,-0.7393,-90.3518
...,...,...
471,8.9167,38.6167
472,33.1925,73.1413
473,-19.8436,34.8389
474,43.2686,-7.5406


In [4]:
# !jupyter nbextension enable --py gmaps

In [26]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig
# 4b. Display the figure


Figure(layout=FigureLayout(height='420px'))

In [36]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City'] == 'Saint Anthony']
vacation_end = vacation_df.loc[vacation_df['City'] == 'Saint Anthony']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Golden']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Saint George'] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Eureka'] 

vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
294,400,Saint Anthony,US,88.75,clear sky,45.0205,-93.218,"Holiday Inn Express Roseville-St. Paul, an IHG..."


In [40]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
# start = locations[["Lat","Lng"]][(vacation_df["City"] == "Minneapolis")].to_numpy()
start = vacation_start[["Lat", "Lng"]].to_numpy()[0]
end = vacation_end[["Lat", "Lng"]].to_numpy()[0]
stop1 = vacation_stop1[["Lat", "Lng"]].to_numpy()[0]
stop2 = vacation_stop2[["Lat", "Lng"]].to_numpy()[0]
stop3 = vacation_stop3[["Lat", "Lng"]].to_numpy()[0]

stop3

array([  40.8021, -124.1637])

In [49]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
roundtrip_to_Minneapolis = gmaps.directions_layer(start, end,
                            waypoints = [stop1, stop2, stop3],
                            show_markers=True, 
                            stroke_color='red', stroke_weight=3.0, stroke_opacity=1.0,
                            travel_mode = "DRIVING")
fig.add_layer(roundtrip_to_Minneapolis)
fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,400,Saint Anthony,US,88.75,clear sky,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
1,198,Golden,US,85.48,scattered clouds,39.7555,-105.2211,"The Golden Hotel, Ascend Hotel Collection"
2,12,Saint George,US,85.53,few clouds,37.1041,-113.5841,Best Western Plus Abbey Inn
3,635,Eureka,US,68.13,scattered clouds,40.8021,-124.1637,Eureka Inn Trademark By Wyndham


In [54]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig

IndexError: list index out of range